<a href="https://colab.research.google.com/github/Tina-223/DBscan/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

df = pd.read_csv('all_drinks.csv')

df = df.drop(['strCategory','dateModified','strDrinkThumb', 'strGlass', 'strIBA', 'strInstructions', 'strMeasure1', 'strMeasure2', 'strMeasure3', 'strMeasure4', 'strMeasure5', 'strMeasure6', 'strMeasure7', 'strMeasure8',
                          'strMeasure9', 'strMeasure10', 'strMeasure11', 'strMeasure12', 'strMeasure13', 'strMeasure14', 'strMeasure15', 'strVideo'], axis=1)
df.head()

,Unnamed: 0,strDrink,idDrink,strAlcoholic,strIngredient1,strIngredient10,strIngredient11,strIngredient12,strIngredient13,strIngredient14,strIngredient15,strIngredient2,strIngredient3,strIngredient4,strIngredient5,strIngredient6,strIngredient7,strIngredient8,strIngredient9
0,0,'57 Chevy with a White License Plate,14029,Alcoholic,Creme de Cacao,NaN,NaN,NaN,NaN,NaN,NaN,Vodka,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1-900-FUK-MEUP,15395,Alcoholic,Absolut Kurant,NaN,NaN,NaN,NaN,NaN,NaN,Grand Marnier,Chambord raspberry liqueur,Midori melon liqueur,Malibu rum,Amaretto,Cranberry juice,Pineapple juice,NaN
2,2,110 in the shade,15423,Alcoholic,Lager,NaN,NaN,NaN,NaN,NaN,NaN,Tequila,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,151 Florida Bushwacker,14588,Alcoholic,Malibu rum,NaN,NaN,NaN,NaN,NaN,NaN,Light rum,151 proof rum,Dark Creme de Cacao,Cointreau,Milk,Coconut liqueur,Vanilla ice-cream,NaN
4,4,155 Belmont,15346,Alcoholic,Dark rum,NaN,NaN,NaN,NaN,NaN,NaN,Light rum,Vodka,Orange juice,NaN,NaN,NaN,NaN,NaN


In [2]:
from sklearn.cluster import DBSCAN

alcoholic = {'Alcoholic' : 1, 'Optional alcohol': 0, 'Non alcoholic': -1}
df['strAlcoholic'] = df['strAlcoholic'].map(alcoholic)

ingredients = {}
initial_data = [0 for i in range(df.shape[0])]

for row in range(df.shape[0]):
  for i in range(1, 16):
    column = 'strIngredient'+ str(i)
    ingr = df.loc[row][column]
    if str(ingr) == '' or str(ingr) == 'nan':
      continue
    if not ingr in ingredients :
      ingredients[ingr] = initial_data.copy()
    ingredients[ingr][row] = 1


df = pd.concat([df, pd.DataFrame(ingredients)], axis=1)
df = df.drop(['strIngredient'+str(i) for i in range(1, 16)], axis=1)
df.head()

,Unnamed: 0,strDrink,idDrink,strAlcoholic,Creme de Cacao,Vodka,Absolut Kurant,Grand Marnier,Chambord raspberry liqueur,Midori melon liqueur,...,Aquavit,Fresca,Yukon Jack,Agave syrup,Tomato Juice,Maple syrup,Lillet Blanc,Limeade,White Rum,Zima
0,0,'57 Chevy with a White License Plate,14029,1.0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1-900-FUK-MEUP,15395,1.0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,2,110 in the shade,15423,1.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,151 Florida Bushwacker,14588,1.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,155 Belmont,15346,1.0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df_train = df.drop(df.columns[[0,1,2,3]], axis=1)
df_train.head()
clustering = DBSCAN(eps=1, min_samples=4).fit(df_train)
df_train.loc[:,'Cluster'] = clustering.labels_
df_train.Cluster.value_counts().to_frame()

,Cluster
-1,514
1,10
4,5
2,5
0,4
3,4
5,4


curse of dimensionality

dimension이 너무 크고 각 feature의 데이터가 binary해서 올바르게 군집화가 발생하지 않는다.

높은 차원에서는 Euclidean Distance 으로 구하는 거리가 커서 군집화 시키기 쉽지 않다.